# Introdução

Este notebook apresenta a rotina de treinamento e validação aplicada à 5 CNNs distintas, considerando a tarefa do reconhecimento de expressões facias de dor em recém-nascidos.

As imagens consumidas por este script foram antes processadas pela rotina apresentada no arquivo Prepara_imagens.ipynb

As métricas de desempenho obtidas com a execução deste notebook foram, posteriormente, processadas pelo arquivo ANOVA.ipynb

Após a execução do presente notebook, os modelos resultantes foram submetidos ao método Grad-CAM por meio do script apresentado em GradCAM.ipynb  

# Configuração Inicial

Modelo a ser treinado

Opções:


* vgg16: VGG-16 pré-treinada no banco de imagens de face VGGFace 
* resnet50: ResNet50 pré-treinada no banco de imagens de face VGGFace2 
* senet50: SENet50 pré-treinada no banco de imagens de face VGGFace2 
* incepv3: Inception-V3 pré-treinada no base ImageNet
* ncnn: Neonatal Convolutional Neural Network (N-CNN) sem treinamento prévio 



In [ ]:
## SELECIONE UM MODEL_NAME ##

model_name='vgg16'
#model_name='resnet50'
#model_name='senet50'
#model_name='incepv3'
#model_name='ncnn'

Bibliotecas

In [ ]:
import os
import os.path as osp
import sys
import cv2
import numpy as np
from PIL import Image
import pandas as pd
from shutil import copyfile, copytree
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_curve, f1_score, roc_auc_score,confusion_matrix, recall_score, precision_score
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Flatten, Dropout, Input
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.losses import CategoricalCrossentropy, BinaryCrossentropy
from tensorflow.keras.regularizers import l1
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras import layers

Hiperparâmetros

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 16
LEARNING_RATE = 1e-4
LEARNING_RATE_ft = 1e-6    # _ft = fine-tuning
CHANNELS = 3 
REGULARIZER = l1(5e-4)
BUFFER_SIZE = 2048
EPOCHS = 1000
OPTIMIZER = RMSprop(learning_rate=LEARNING_RATE)
OPTIMIZER_ft = RMSprop(learning_rate=LEARNING_RATE_ft)  # _ft = fine-tuning
LOSS = tf.keras.losses.SparseCategoricalCrossentropy() 

if model_name == "incepv3":

  IMG_HEIGHT = 299 
  IMG_WIDTH = 299

elif model_name == "ncnn":

  IMG_HEIGHT = 120
  IMG_WIDTH = 120

else:
  IMG_HEIGHT = 224
  IMG_WIDTH = 224 

Biblioteca dos modelos pré-treinados em imagens de face: VGG-16, ResNet50 e SENet50

In [ ]:
#Instalação da biblioteca
!pip install git+https://github.com/rcmalli/keras-vggface.git --quiet
!pip install keras_applications --no-deps

#ajuste de dependências do arquivo "models.py"
filename = "/usr/local/lib/python" + sys.version[0:3] + "/dist-packages/keras_vggface/models.py"
text = open(filename).read()
open(filename, "w+").write(text.replace('keras.engine.topology', 'tensorflow.keras.utils'))

#importando biblioteca instalada
from keras_vggface.vggface import VGGFace
from keras_vggface import utils


Função que instancia modelos pré-treinados

In [ ]:
# definição de função para instanciar modelo pré-treinado
def base_model(model_name):
  """
  **valores de model_name**

  vgg16: VGG-16 pré-treinada no banco de imagens de face VGGFace (modelo disponibilizado por R.C. Malli)
  resnet50: ResNet50 pré-treinada no banco de imagens de face VGGFace2 (modelo disponibilizado por R.C. Malli)
  senet50: SENet50 pré-treinada no banco de imagens de face VGGFace2 (modelo disponibilizado por R.C. Malli)
  incepV3: Inception-V3 pré-treinada no base ImageNet (modelo disponibilizado pelo TensorFlow)

  """
  if model_name == "vgg16":
    PRE_TRAINED = VGGFace(include_top=False, input_shape=(224,224,3), model= model_name, pooling=None)

  elif model_name in ['resnet50','senet50']:
    PRE_TRAINED = VGGFace(include_top=False, input_shape=(224,224,3), model= model_name, pooling='avg')

  elif model_name == "incepv3":
    PRE_TRAINED = tf.keras.applications.inception_v3.InceptionV3(
    include_top=False, weights='imagenet', input_tensor=None,
    input_shape=input_size, pooling='avg')

  return PRE_TRAINED

Implementação da N-CNN no TensorFlow

A Neonatal Convolutional Neural Network (N-CNN) é a primeira CNN desenvolvida especificamente para o reconhecimento da dor neonatal em imagens de face. O modelo foi proposto por Zamzmi et al. (2019) em "Pain assessment from facial expression: Neonatal convolutional neural
network (N-CNN)", que está disponível em https://ieeexplore.ieee.org/abstract/document/8851879

Até o momento do desenvolvimento do presente trabalho, não há versão do modelo disponível *online*, portanto, realizou-se aqui sua implementação no TensorFlow, seguindo a descrição apresentada em Zamzmi et al. (2019). No entanto, a última cada do modelo foi alterada: de uma saída sigmoide com um neurônio, passou a ser uma saída softmax com 2 neurônios. Este ajuste foi feito para a aplicação do Grad-CAM, encontrada no arquivo GradCAM.ipynb

In [ ]:
# função que instancia uma N-CNN
def NCNN():
  
  input = tf.keras.layers.Input(shape = (120,120,3), name='input')

  # Branch 1
  maxpool_1 = tf.keras.layers.MaxPooling2D(pool_size=10, strides=10,
                                          padding='same', name='maxpool_10x10') (input)

  # Branch 2
  conv_1 = tf.keras.layers.Conv2D(filters=64, kernel_size=5, strides=(1,1),
                                  padding='same', kernel_initializer=tf.keras.initializers.glorot_normal(), 
                                  name='conv_5x5') (input)

  leaky_relu_1 = tf.keras.layers.LeakyReLU(alpha = 0.01, name='leaky_1') (conv_1)                               

  maxpool_2 = tf.keras.layers.MaxPooling2D(pool_size=3, strides=3,
                                          padding='same',name='maxpool_3x3') (leaky_relu_1)

  conv_2 = tf.keras.layers.Conv2D(filters=64,kernel_size=2,strides=(1,1),padding='same',
                                  kernel_initializer=tf.keras.initializers.glorot_normal(),name='conv_2x2') (maxpool_2)

  leaky_relu_2 = tf.keras.layers.LeakyReLU(alpha = 0.01, name='leaky_2') (conv_2)                               

  maxpool_3 = tf.keras.layers.MaxPooling2D(pool_size=3,strides=3,
                                          padding='same',name='maxpool_3x3_') (leaky_relu_2)

  resize = tf.keras.layers.experimental.preprocessing.Resizing(12,12) (maxpool_3)

  dropout_1 = tf.keras.layers.Dropout(0.1) (resize)

  # Branch 3
  conv_3 = tf.keras.layers.Conv2D(filters=64,kernel_size=5,strides=(1,1),padding='same',
                                  kernel_initializer=tf.keras.initializers.glorot_normal(),name='conv_5x5_') (input)

  leaky_relu_3 = tf.keras.layers.LeakyReLU(alpha = 0.01, name='leaky_3') (conv_3)                               

  maxpool_4 = tf.keras.layers.MaxPooling2D(pool_size=10,strides=10,
                                          padding='same',name='maxpool_10x10_') (leaky_relu_3)

  dropout_2 = tf.keras.layers.Dropout(0.1) (maxpool_4)

  # Merge Branch
  merge = tf.keras.layers.concatenate([maxpool_1,dropout_1,dropout_2],axis=3)

  conv_4 = tf.keras.layers.Conv2D(filters=64,kernel_size=2,strides=(1,1),padding='same',
                                  kernel_initializer=tf.keras.initializers.glorot_normal(),name='conv_2x2_',
                                  activation='relu') (merge)

  maxpool_5 = tf.keras.layers.MaxPooling2D(pool_size=2,strides=2, padding='same',name='maxpool_2x2_') (conv_4)

  flatten = tf.keras.layers.Flatten()(maxpool_5)

  fc1 = tf.keras.layers.Dense(units=8,kernel_initializer=tf.keras.initializers.glorot_normal(),
                              activation='relu',name='FC1', kernel_regularizer='l2') (flatten)

  dropout_3 = tf.keras.layers.Dropout(0.1) (fc1)

  output = tf.keras.layers.Dense(units=2,kernel_initializer=tf.keras.initializers.glorot_normal(),
                                activation='softmax',name='output') (dropout_3)

  # NCNN
  ncnn = tf.keras.models.Model(inputs=input,outputs=output)
  return ncnn

Função de pré-processamento das imagens

In [ ]:
#pré-processamento para o modelo Inception-V3
preprocess_incepV3 = tf.keras.applications.inception_v3.preprocess_input

from tensorflow.keras import backend


# função de pré-processamento usada no loop de treino
def preprocess_input(x, model_name):

  # Inception-V3
  if model_name=="incepV3":

    return preprocess_incepV3(x)

  # N-CNN
  elif model_name == "ncnn":

    return x / 255

  # VGG-16, ResNet50 e SENet50 --> conforme o repositório de R. C. Malli
  else:
  
    # formato da imagem (canal na primeira ou na última dimensão)
    data_format = backend.image_data_format()
    
    if data_format == 'channels_first':
      # 'RGB'->'BGR'
      if backend.ndim(x) == 3:
        x = x[::-1, ...]
      else:
        x = x[:, ::-1, ...]
    else:
      # 'RGB'->'BGR'
      x = x[..., ::-1]

    # valor médio dos canais da imagem, considerando a base do treinamento original
    # de cada modelo

    if model_name =="vgg16":
      mean = [93.5940, 104.7624, 129.1863]  #vggface  --> VGG-16

    else:
      mean=[91.4953, 103.8827, 131.0912]   #vggface2  --> ResNet50, SENet50

    # desvio padrão da base
    std = None

    # tensor médio
    mean_tensor = backend.constant(-np.array(mean))

    # Centralização da imagem em zero, utilizando o tensor médio da base
    if backend.dtype(x) != backend.dtype(mean_tensor):
      x = backend.bias_add(
          x, backend.cast(mean_tensor, backend.dtype(x)), data_format=data_format)
    else:
      x = backend.bias_add(x, mean_tensor, data_format)
      
    if std is not None:
      std_tensor = backend.constant(np.array(std), dtype=backend.dtype(x))
      if data_format == 'channels_first':
        std_tensor = backend.reshape(std_tensor, (-1, 1, 1))

      x /= std_tensor
    return x

Funções auxiliares

In [ ]:
#Criação de Log
def log_print(texto):

    print(texto)
    with open('./log.txt',"a") as txt:
        txt.write(texto + '\n')


# Carregando imagens

O presente trabalho utilizou a união de duas bases de dados distintas.
Antes da execução do script a seguir, as imagens de cada base foram organizadas em pastas de acordo com o indivíduo da amostra. Além disso, as imagens originais e as imagens provenientes do processo de Data Augmentation foram organizadas em diretórios distintos

Por exemplo:

* Imagens Originais
  *   Base A
    *   ID_01
        * ID01_imagem_1
        * ID01_imagem_2
        * ...
        * ID01_imagem_*N*

    * ID_02
    * ID_03
    * ...
    * ID_*N*

  * Base B

* Imagens Data Augmentation
  *   Base A
    *   ID_01
        * ID01_imagemAUG_1
        * ID01_imagemAUG_2
        * ...
        * ID01_imagemAUG_*N*

    * ID_02
    * ID_03
    * ...
    * ID_*N*

  * Base B

In [ ]:
# imagens das bases iCOPE e UNIFESP, separadas por subject
!unzip -q ./COPE_UNIFESP_NB.zip 

# imagens provenientes de Data Augmentation das bases iCOPE e UNIFESP, 
# separadas por subject
!unzip -q ./COPE_UNIFESP_NB_aug.zip

# diretórios auxiliares
!mkdir ./History
!mkdir ./ROC
!mkdir ./ConfusionMatrix
!mkdir ./Resultados
!mkdir ./COPE
!mkdir ./UNIFESP
!mkdir ./COPE+UNIFESP
!mkdir ./UC_weights

!mkdir ./COPE+UNIFESP/Fold0
!mkdir ./COPE+UNIFESP/Fold1
!mkdir ./COPE+UNIFESP/Fold2
!mkdir ./COPE+UNIFESP/Fold3
!mkdir ./COPE+UNIFESP/Fold4
!mkdir ./COPE+UNIFESP/Fold5
!mkdir ./COPE+UNIFESP/Fold6
!mkdir ./COPE+UNIFESP/Fold7
!mkdir ./COPE+UNIFESP/Fold8
!mkdir ./COPE+UNIFESP/Fold9


# Treinamento e Teste

In [ ]:
# dict para armazenar os resultados das iterações da validação cruzada
COPE_UNIFESP={}

# versão do modelo
version="_v1_"

# qtd de camadas para fine-tuning

if model_name ="vgg16":
  fine_tune_at = 14 #grupos conv4 e conv5 da Vgg-16

elif model_name ="resnet50":
  fine_tune_at = 36  # grupo conv5 da ResNet50

elif model_name = "senet50":
  fine_tune_at = 59 # grupo conv5 da SENet50

elif model_name = "incepv3":
  fine_tune_at = 68 # 18 camadas conv. da Inception-V3

else:
  fine_tune_at = None  # não haverá fine-tuning

# diretórios das imagens originais
path_cope = './COPE_UNIFESP_NB/COPE_NB'
path_unifesp = './COPE_UNIFESP_NB/UNIFESP_NB'

# diretórios das imagens criadas pelo Data Augmentation
path_cope_aug = './COPE_UNIFESP_NB_aug/COPE_NB_aug'
path_unifesp_aug = './COPE_UNIFESP_NB_aug/UNIFESP_NB_aug'

# arquivos .npy com os IDs dos subjects destinados a treinamento e teste em 
# cada iteração da validação cruzada
treino = np.load('./NB_KFold10_Train.npy',allow_pickle=True)
teste = np.load('./NB_KFold10_Test.npy',allow_pickle=True)

#bancos de imagens
DBS = ['COPE+UNIFESP']

# loop de treino
for DB in DBS:

  # 10 repetições por banco de imagens
  for contador in range(10):

    # Inicio do Log do treinamento
    log_print(DB+' Contagem: '+ str(contador))
    
    # instanciando modelo

    if model_name in ['vgg16','incepv3']:

      # instanciando o modelo pré-treinado (sem camadas FC)
      PRE_TRAINED =  base_model(model_name)    
      PRE_TRAINED.trainable = False

      # inclusão do novo conjunto de camadas FC no final do modelo
      # duas camadas FC com 512 neuronios e uma saída softmax de 2 neurônios
      inputs = PRE_TRAINED.input
      x = Flatten()(PRE_TRAINED.output)
      x = Dense(512,activation='relu',kernel_regularizer=REGULARIZER, dtype='float32')(x)
      x = Dropout(0.5)(x) 
      x = Dense(512,activation='relu',kernel_regularizer=REGULARIZER, dtype='float32')(x)
      x = Dropout(0.5)(x)
      outputs = Dense(2,activation='softmax', dtype='float32')(x)
      # novo modelo
      FT_MODEL = tf.keras.Model(inputs,outputs)

    elif model_name in ['resnet50','senet50']:

      # instanciando o modelo pré-treinado (sem camadas FC)
      PRE_TRAINED =  base_model(model_name)    
      PRE_TRAINED.trainable = False

      # inclusão do novo conjunto de camadas FC no final do modelo
      # uma camada FC com 1000 neuronios e uma saída softmax de 2 neurônios
      inputs = PRE_TRAINED.input 
      x = Dense(1000,activation='relu',kernel_regularizer=REGULARIZER, dtype='float32')(PRE_TRAINED.output)
      x = Dropout(0.5)(x)
      outputs = Dense(2,activation='softmax', dtype='float32')(x)
      # novo modelo
      FT_MODEL = tf.keras.Model(inputs,outputs)

    else:
      FT_MODEL = NCNN()

    

    #criaçãos dos diretórios das imagens
    if os.path.isdir('./dataset'):
      !rm -r ./dataset
      
    !mkdir ./dataset

    !mkdir ./dataset/train

    !mkdir ./dataset/train/COPE+UNIFESP
    !mkdir ./dataset/train/COPE+UNIFESP/dor
    !mkdir ./dataset/train/COPE+UNIFESP/sem_dor

    !mkdir ./dataset/validation
    !mkdir ./dataset/validation/dor
    !mkdir ./dataset/validation/sem_dor

    # separaçãos da imagens de treinamento e teste
    x_train, x_test = treino[contador].tolist(), teste[contador].tolist()    

    # abastecimento diretórios de imagens de treinamento
    for i in range(len(x_train)):

      # para cada subject de treinamento em x_train
      # avalia-se sua base de origem é UNIFESP (U) ou iCOPE (C)
      base = x_train[i][0]  # U ou C

      # conforme a base, são determinados os diretórios para coleta das imagens
      # originais e das imagens provenientes do Data Augmentation  
      path = path_cope if base == 'C' else path_unifesp
      path_aug = path_cope_aug if base == 'C' else path_unifesp_aug      
      

      # Movimentação das imagens do subject para o diretório de imagens de treinamento    
      src = osp.join(path,x_train[i])
      src_aug = osp.join(path_aug,x_train[i]+"_aug")

      for img in os.listdir(osp.join(src,"Dor")):
        copyfile(osp.join(src,"Dor",img),osp.join('./dataset/train/COPE+UNIFESP/dor',img))

      for img in os.listdir(osp.join(src_aug,"Dor")):
        copyfile(osp.join(src_aug,"Dor",img),osp.join('./dataset/train/COPE+UNIFESP/dor',img))

      for img in os.listdir(osp.join(src,"Sem_dor")):
        copyfile(osp.join(src,"Sem_dor",img),osp.join('./dataset/train/COPE+UNIFESP/sem_dor',img))

      for img in os.listdir(osp.join(src_aug,"Sem_dor")):
        copyfile(osp.join(src_aug,"Sem_dor",img),osp.join('./dataset/train/COPE+UNIFESP/sem_dor',img))


    # abastecimento diretórios de imagens de teste
    for i in range(len(x_test)):

      # para cada subject de teste em x_test
      # avalia-se sua base de origem é UNIFESP (U) ou iCOPE (C)
      base = x_test[i][0]  # U ou C

      # conforme a base, são determinados os diretórios para coleta das imagens
      # originais e das imagens provenientes do Data Augmentation  
      path = path_cope if base == 'C' else path_unifesp
      src = osp.join(path,x_test[i])

      # Movimentação das imagens do subject para o diretório de imagens de teste  
      for img in os.listdir(osp.join(src,"Dor")):
        copyfile(osp.join(src,"Dor",img),osp.join('./dataset/validation/dor',img))

      for img in os.listdir(osp.join(src,"Sem_dor")):
        copyfile(osp.join(src,"Sem_dor",img),osp.join('./dataset/validation/sem_dor',img))
            

    #criação dos datasets (objeto TensorFlow) de treinamento e teste
    PATH = './dataset/'

    train_ds = tf.keras.preprocessing.image_dataset_from_directory(PATH+'train/'+DB,
                                                                  image_size=(IMG_HEIGHT, IMG_WIDTH),
                                                                  batch_size=BATCH_SIZE,
                                                                  label_mode='binary')

    val_ds = tf.keras.preprocessing.image_dataset_from_directory(PATH+'validation',
                                                                image_size=(IMG_HEIGHT, IMG_WIDTH),
                                                                batch_size=BATCH_SIZE,
                                                                label_mode='binary')

    #aplicação da função de pré-processamento nas imagens dos datasets
    train_ds = train_ds.map(lambda x, y: (preprocess_input(x, model_name), y), num_parallel_calls=AUTOTUNE)
    val_ds = val_ds.map(lambda x, y: (preprocess_input(x, model_name), y), num_parallel_calls=AUTOTUNE)

    train_ds = train_ds.cache().shuffle(BUFFER_SIZE).prefetch(AUTOTUNE)
    val_ds = val_ds.cache().prefetch(AUTOTUNE)

    # Diretório para gravação do modelo originado pela iteração
    fold = './'+DB+'/Fold' + str(contador) + "/"

    # Callback para gravação do modelo de menor erro durante o treinamento. Apenas os pesos são salvos.
    check = ModelCheckpoint(fold + model_name + version + DB + "_" + str(contador) + '_weights.h5',
                        monitor='val_loss',
                        verbose=0,
                        save_best_only=True,
                        save_weights_only=True)

    # Critério de parada do treinamento: 
    # - 5 épocas consecutivas sem redução do erro p/ modelos pré-treinados
    # - 10 épocas consecutivas sem redução do erro p/ N-CNN

    patience = 5 if fine_tune_at else 10
    stop = EarlyStopping(monitor='val_loss',
                         patience=patience,
                         verbose=0)

    # compilando o modelo...
    FT_MODEL.compile(optimizer=OPTIMIZER,
                 loss=LOSS,
                 metrics='acc')
    
    # Treinamento da CNN
    history = FT_MODEL.fit(train_ds,
                          validation_data=val_ds,
                          epochs=EPOCHS,
                          callbacks=[check, stop],
                          verbose=0
                          )
    
    ft=False  ## variável auxiliar que será utilizada na finalização do log (indica se houve fine-tuning ou não)

    ## FINE TUNING
    # se o modelo for  pré-treinado e seu novo treinamento foi encerrado pelo critério das 5 épocas consecutivas, 
    # isto é,o treinamento não alcançou o total de épocas, inicia-se o fine-tuning
    # *OBS: não há fine-tuning para a N-CNN!
    if (len(history.history['val_loss']) < EPOCHS) and fine_tune_at:
      print("Fine Tuning!")

      # Inicia-se o fine-tuning
      ft=True
      
      # Carregando o modelo de melhor acurácia obitido na etapa anterior
      FT_MODEL.load_weights(fold + model_name + version + DB + "_" + str(contador) + '_weights.h5')

      # Todas as camadas são liberadas para treinamento
      FT_MODEL.trainable = True

      # As camadas anteriores a camada `fine_tune_at` são congeladas, ou seja, 
      # não sofrerão o ajuste de pesos
      for layer in FT_MODEL.layers[:-fine_tune_at]:
        layer.trainable =  False

      # A partir da camada 'fine_tune_at' até a saída do modelo, todas as camadas
      # sofrerão o ajuste de pesos, com exceção das camadas de BatchNormalization
      # (caso o modelo tenha), seguindo recomendação do TensorFlow
      for layer in FT_MODEL.layers[-fine_tune_at:]:
        if isinstance(layer, layers.BatchNormalization):
          layer.trainable = False

      # Critério de parada do treinamento durante o fine-tuning: 10 épocas consecutivas sem redução do erro
      stop_ft = EarlyStopping(monitor='val_loss',
                              patience=10,
                              verbose=0)
      
      # Callback para gravação do modelo de menor erro durante o treinamento. Apenas os pesos são salvos.
      check_ft = ModelCheckpoint(fold + 'FT_' + model_name + version + DB+"_" + str(contador) + '_weights.h5',
                        monitor='val_loss',
                        verbose=0,
                        save_best_only=True,
                        save_weights_only=True)

      # compilando o modelo...
      FT_MODEL.compile(optimizer=OPTIMIZER_ft,
                 loss=LOSS,
                 metrics='acc')
      
      # qtd de épocas de fine-tuning.
      # selecionou-se um número elevado pois o objetivo é que o treinamento
      # seja encerrado pelo critério das 10 épocas
      fine_tune_epochs = 100

      # ajuste do índice da época para registro do histórico de treinamento.
      # Este ajuste evita a sobrescrição no histórico do treinamento anterior ao fine-tuning.
      total_epochs =  EPOCHS + fine_tune_epochs

      # novo treinamento do modelo
      history_fine = FT_MODEL.fit(train_ds,
                           validation_data=val_ds,
                           epochs=total_epochs,
                           initial_epoch=history.epoch[-1],
                           callbacks=[check_ft, stop_ft],
                           verbose=0
                          )


    #----------------------------##
    ## Testando a CNN ##

    # coletando as classes das imagens
    y_true = []
    for element in val_ds.unbatch():
        y_true.append(np.array(element[1]))
    
    # carregando a melhor versão do modelo na iteração da validação cruzada
    try:
      # tenta carregar umaa versão proveniente do fine-tuning
      FT_MODEL.load_weights(fold +'FT_' + model_name + version + DB+"_" + str(contador) + '_weights.h5')

    except:
      # se não houver, é utilizada uma versão sem fine-tuning
      FT_MODEL.load_weights(fold + model_name + version + DB + "_" + str(contador) + '_weights.h5')

    # a CNN classifica as imagens reservadas para teste
    y_pred = FT_MODEL.predict(val_ds)

    # tratamento da classificação softmax
    y_pred = [0 if np.argmax(y) == 0 else 1 for y in y_pred]

    # metricas de desempenho
    acc = accuracy_score(y_true,y_pred)
    pr = precision_score(y_true,y_pred)
    rc = recall_score(y_true,y_pred)
    f1 = f1_score(y_true,y_pred)
    AUC = roc_auc_score(y_true,y_pred)

    # curva ROC
    fpr, tpr, th = roc_curve(y_true,y_pred)
    roc_dict={'fpr':fpr, 'tpr':tpr, 'th': th}
    roc_csv = "./ROC/ROC_"+ model_name + version + DB + "_" + str(contador)+".csv"
    pd.DataFrame.from_dict(roc_dict).to_csv(roc_csv,index=False,sep=";",decimal=',')

    # matriz de confusao
    CMatrix_file = './ConfusionMatrix/CM_'+ model_name + version + DB + "_" + str(contador)+".npy"
    cmatrix = confusion_matrix(y_true,y_pred)
    np.save(CMatrix_file,cmatrix)

    # quantidade de épocas
    epochs_hist = len(history.history['val_loss'])

    # quantidade de épocas em fine-tuning (se houver)
    if ft:
      epochs_ft = len(history_fine.history['val_loss']) 
    else:
      epochs_ft= 0
    
    # registro em log das métricas de desempenho e quantidade de épocas
    log_print(f"Acuracia: {acc:.4f}\nPrecision: {pr:.4f}\nRecall: {rc:.4f}\nF1: {f1:.4f}\nAUC: {AUC:.4f}\nEpocas: {epochs_hist}\nEpocas_FT: {epochs_ft}")

    
    # histórico de treinamento
    csv_name = "./History/history_" + model_name+version+DB+"_"+str(contador)+".csv"
  
    if ft:
      pd.concat([pd.DataFrame.from_dict(history.history),pd.DataFrame.from_dict(history_fine.history)]).to_csv(csv_name,index=False,sep=";",decimal=',')
    else:
      pd.DataFrame.from_dict(history.history).to_csv(csv_name,index=False,sep=";",decimal=',')

# fim do loop de treinamento e validação
print("Fim!!!")
